In [29]:
import pandas as pd

df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
print(df.shape)  # how many rows & columns

(7043, 21)


In [30]:
# print(df.head())  # see first few rows
# print(df.info())  # check data types
# print(df['Churn'].value_counts())  # see churn vs non-churn

## Clean the data

In [14]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

In [31]:
df['Churn']

0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7038     No
7039     No
7040     No
7041    Yes
7042     No
Name: Churn, Length: 7043, dtype: object

### Convert categorical variables

columns like gender, Contract, PaymentMethod -> one-hot encode.

In [16]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('customerID')  # drop ID
categorical_cols.remove('Churn')       # target

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

### Encode the target

In [33]:
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
# df['Churn'][:50]

In [36]:
X = df.drop(['customerID', 'Churn'], axis=1)
y = df['Churn']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)